### 8.1 Implement a stack with max API

Design a stack that includes a max operation, in addtion to push and pop, The max method should return the maximum value stored in the stack.

Hint: Use addtional storage to track the maximum value.

### Preliminary thoughts

The max can only change whenever *push* and *pop* are called.  I should be able to use the built in Python list, along with *append* and *pop* to implement a stack.

#### Solution #1

In [1]:
class Stack():
    """A custom stack class, with a 'max' data member"""
    def __init__(self):
        """Our two class data members"""
        self.data = []
        self.max = None
        
    def push(self, value):
        """pushes a value onto the 'top' of the stack, updating the 'max' value"""
        old_max = self.max
        if not self.max or self.max < value:
            old_max = self.max
            self.max = value
        self.data.append((value, old_max))
        
    def pop(self):
        """returns the value at the 'top' of the stack, updating the 'max' value"""
        if len(self.data) == 0:
            return None
        value, old_max = self.data.pop()
        self.max = old_max
        return value
    
    def is_empty(self):
        """Prevents popping from an empty stack, to be used in while loops"""
        if len(self.data) == 0:
            return True
        return False
    

# Demonstration    
s = Stack()
# Build a stack
show = lambda s: print("stack: {}, max: {}".format(s.data, s.max))

print("Build the stack")
show(s)
for item in [1, 3, 4, 2, 4]:
    s.push(item)
    show(s)

print("\nConsume the stack")
# Consume the stack
while not s.is_empty():
    show(s)
    print(s.pop())
show(s)


Build the stack
stack: [], max: None
stack: [(1, None)], max: 1
stack: [(1, None), (3, 1)], max: 3
stack: [(1, None), (3, 1), (4, 3)], max: 4
stack: [(1, None), (3, 1), (4, 3), (2, 4)], max: 4
stack: [(1, None), (3, 1), (4, 3), (2, 4), (4, 4)], max: 4

Consume the stack
stack: [(1, None), (3, 1), (4, 3), (2, 4), (4, 4)], max: 4
4
stack: [(1, None), (3, 1), (4, 3), (2, 4)], max: 4
2
stack: [(1, None), (3, 1), (4, 3)], max: 4
4
stack: [(1, None), (3, 1)], max: 3
3
stack: [(1, None)], max: 1
1
stack: [], max: None


### Remarks:

I used a 2-tuple for each item in my self.data array.  The first item in the tuple is the value, and the second item is the max value during the time it was pushed.  The second value allows me to restore the max for the entire stack while popping.  Pushing and popping should be O(1) operations, the space required should be an addtional O(n).  After reviewing the solutions in the book, it seems we can save lots of space by instead using a Python dictionary.  Let's try to save that space with another solution.

#### Solution #2

In [2]:
class Stack():
    """A custom stack class, with a 'max' data member"""
    def __init__(self):
        """Our two class data members"""
        self.data = []
        self.max = {}
        self.abs_max = None
        
    def push(self, value):
        """pushes a value onto the 'top' of the stack, updating the 'max' dict and 
        perhaps the abs_max value"""
        if value not in self.max:
            if not self.abs_max or self.abs_max < value:
                self.abs_max = value
            self.max[value] = 1
        else:
            self.max[value] += 1
        self.data.append(value)
        
    def pop(self):
        """returns the value at the 'top' of the stack, updating the 'max' value"""
        if len(self.data) == 0:
            return None
        value = self.data.pop()
        self.max[value] -= 1
        if self.max[value] <= 0:
            del self.max[value]
            if not self.is_empty():
                self.abs_max = max(self.max, key=int)
            else:
                self.abs_max = None
        return value
    
    def is_empty(self):
        """Prevents popping from an empty stack, to be used in while loops"""
        if len(self.data) == 0:
            return True
        return False
    

# Demonstration    
s = Stack()
# Build a stack
show = lambda s: print("stack: {}, abs_max: {}, max: {}".format(s.data, s.abs_max, s.max))

print("Build the stack")
show(s)
for item in [1, 3, 4, 2, 4]:
    s.push(item)
    show(s)

print("\nConsume the stack")
# Consume the stack
while not s.is_empty():
    show(s)
    print(s.pop())
show(s)

Build the stack
stack: [], abs_max: None, max: {}
stack: [1], abs_max: 1, max: {1: 1}
stack: [1, 3], abs_max: 3, max: {1: 1, 3: 1}
stack: [1, 3, 4], abs_max: 4, max: {1: 1, 3: 1, 4: 1}
stack: [1, 3, 4, 2], abs_max: 4, max: {1: 1, 3: 1, 4: 1, 2: 1}
stack: [1, 3, 4, 2, 4], abs_max: 4, max: {1: 1, 3: 1, 4: 2, 2: 1}

Consume the stack
stack: [1, 3, 4, 2, 4], abs_max: 4, max: {1: 1, 3: 1, 4: 2, 2: 1}
4
stack: [1, 3, 4, 2], abs_max: 4, max: {1: 1, 3: 1, 4: 1, 2: 1}
2
stack: [1, 3, 4], abs_max: 4, max: {1: 1, 3: 1, 4: 1}
4
stack: [1, 3], abs_max: 3, max: {1: 1, 3: 1}
3
stack: [1], abs_max: 1, max: {1: 1}
1
stack: [], abs_max: None, max: {}


### Remarks

I can save even more space by using a dictionary, and keeping an eye on the highest value in the dictionary.  Now the extra O(n) space is condensed to a dictionary of key,value pairs.  There's a little bit of cost in time to maintain the dictionary, and re-discovering the max key in the dictionary after a pop could in the worst case be an extra O(n) of time.  This actually could be improved even further by using a list instead of a dict.

#### Solution #3

In [3]:
class Stack():
    """A custom stack class, with a 'max' data member"""
    def __init__(self):
        """Our two class data members"""
        self.data = []
        self.max = []
        
    def push(self, value):
        """pushes a value onto the 'top' of the stack, updating the 'max' value"""
        if not self.max:
            self.max.append((value, 1))
            self.data.append(value)
            return
        cur_max, cur_count = self.max[-1]
        if value <= cur_max:
            self.max[-1] = (cur_max, cur_count + 1)
        else:
            self.max.append((value, 1))
        self.data.append(value)
        
    def pop(self):
        """returns the value at the 'top' of the stack, updating the 'max' value"""
        if len(self.data) == 0:
            return None
        cur_max, cur_count = self.max[-1]
        if cur_count > 1:
            self.max[-1] = (cur_max, cur_count - 1)
        else:
            self.max = self.max[:-1]
        value = self.data.pop()

        return value
    
    def is_empty(self):
        """Prevents popping from an empty stack, to be used in while loops"""
        if len(self.data) == 0:
            return True
        return False
    
    def get_max(self):
        if not self.max:
            return None
        else:
            return self.max[-1][0]
    

# Demonstration    
s = Stack()
# Build a stack
show = lambda s: print("stack: {}, max: {}".format(s.data, s.get_max()))

print("Build the stack")
show(s)
for item in [1, 3, 4, 2, 4]:
    s.push(item)
    show(s)

print("\nConsume the stack")
# Consume the stack
while not s.is_empty():
    show(s)
    print(s.pop())
show(s)

Build the stack
stack: [], max: None
stack: [1], max: 1
stack: [1, 3], max: 3
stack: [1, 3, 4], max: 4
stack: [1, 3, 4, 2], max: 4
stack: [1, 3, 4, 2, 4], max: 4

Consume the stack
stack: [1, 3, 4, 2, 4], max: 4
4
stack: [1, 3, 4, 2], max: 4
2
stack: [1, 3, 4], max: 4
4
stack: [1, 3], max: 3
3
stack: [1], max: 1
1
stack: [], max: None


### Concluding remarks

This seems to have a O(1) push and pop time complexity.  It seems to have O(n) storage for the stack and it has worst-case an additional O(n) storage for the max.  However in the best case it has O(1) storage for the max.  Also, it's always an O(1) operation to **get_max**.  This seems to have a best mix of time and space complexity of the three solutions.